In [1]:
import sys
import os

# Add the root directory to Python path
os.chdir(os.path.abspath('..'))



In [3]:
from scripts.data_loader.tmlu_loader import TMLUDataLoader

# TMLU dataset
tmlu_loader = TMLUDataLoader("miulab/tmlu")
tmlu_dataset = tmlu_loader.load_dataset()

Dataset already exists at ./data/eval\miulab/tmlu. Loading from disk...


In [ ]:
def concat_fields(example):
    example['concat_field'] = example['A'] + ' ' + example['B']
    return example

# Apply the function to both 'test' and 'dev' splits in parallel
dataset = tmlu_dataset.map(concat_fields, num_proc=4)

In [21]:
def create_user_prompt(example):
    # Template for the question
    prompt_template = '''
    以下選擇題為出自臺灣的考題，答案為其中一個選項。

    問題:
    {question}

    '''
    # List of possible options (A to F)
    options = ["A", "B", "C", "D", "E", "F"]
    
    # Check if each option exists in the example and construct answer string
    answer_template = ' '.join([f"({option}) {example[option]}" for option in options if example.get(option)])
    
    # Fill the question in the prompt template
    prompt = prompt_template.format(question=example["question"]) + answer_template
    
    # Add the final prompt to 'user_content'
    example['user_content'] = prompt
    
    return example


In [26]:
dataset = tmlu_dataset.map(create_user_prompt, num_proc=2)

In [29]:
len(dataset['test']['user_content'])

2796

In [32]:
dataset['test']['user_content']

['\n    以下選擇題為出自臺灣的考題，答案為其中一個選項。\n\n    問題:\n    閱讀下文，選出依序最適合填入□內的選項：\n甲、我母親和我姑姑一同出洋去，上船的那天她伏在竹床上痛哭，綠衣綠裙上面釘有□□發光的小片子。(張愛玲〈私語〉)\n乙、蝴蝶的本能是吮吸花蜜，女人的愛亦是一種本能：採集所有美好事物引誘自己進入想像，從自身記憶□□□□並且偷摘他人經驗之片段，想像繁殖成更豐饒的想像，織成一張華麗的密網。（簡媜〈母者〉）\n丙、母親是天可汗，當家的天可汗，一家之王，絕對的威權，分配空間與食物的主人。她要我報告的事，或她突如其來要我□□的事，我最好都要知道，所以我在覲見可汗時，不論她問不問我話，我的心中就是會先有腹稿。(鍾文音〈我的天可汗〉)\n\n    (A) 張揚／綢繆未雨／奏疏 (B) 抽搐／煮繭抽絲／奏疏 (C) 張揚／煮繭抽絲／進貢\t (D) 抽搐／綢繆未雨／進貢\t',
 '\n    以下選擇題為出自臺灣的考題，答案為其中一個選項。\n\n    問題:\n    下列各組文句，前後意義最相近的是：\n\n    (A) 聖人欲上民，必以言下之；欲先民，必以身後之／先之，勞之 (B) 不以一己之利為利，而使天下受其利；不以一己之害為害，而使天下釋其害／民之所好好之，民之所惡惡之 (C) 無冥冥之志者，無昭昭之明；無惛惛之事者，無赫赫之功／多聞闕疑，慎言其餘，則寡尤；多見闕殆，慎行其餘，則寡悔 (D) 拔人之城而非攻也，毀人之國而非久也，必以全爭於天下，故兵不頓，而利可全／殺十人十重不義，必有十死罪矣；殺百人百重不義，必有百死罪矣',
 '\n    以下選擇題為出自臺灣的考題，答案為其中一個選項。\n\n    問題:\n    依據下文，最能概括文章內容的標題是：\n法蘭克福是大多數外國旅客到德國的第一站，除了巨大繁忙的法蘭克福國際機場外，這裡也是德國的鐵、公路交通樞紐，以及歐陸的金融中心，更因是歐洲中央銀行的所在地，在世界經濟上的地位可謂舉足輕重。\n因為是銀行城，市區內現代的摩天大樓林立，使法蘭克福的天際線十分絢麗。除了工商業充斥，她也是一座文化古都。從1562年起，法蘭克福一直是神聖羅馬帝國皇帝加冕的地方，崇高的地位延續到該帝國1806年瓦解為止。旅行者可從市中心古色古香的羅馬山廣場，感受歐洲中古風情。\n法蘭克福位於美茵河畔

In [4]:
type(tmlu_dataset)

datasets.dataset_dict.DatasetDict

In [6]:
tmlu_dataset

DatasetDict({
    test: Dataset({
        features: ['id', 'question', 'A', 'B', 'C', 'D', 'E', 'F', 'answer', 'explanation', 'metadata', 'human_evaluation', 'subject'],
        num_rows: 2796
    })
    dev: Dataset({
        features: ['id', 'question', 'A', 'B', 'C', 'D', 'E', 'F', 'answer', 'explanation', 'metadata', 'human_evaluation', 'subject'],
        num_rows: 185
    })
})

In [10]:
tmlu_dataset["test"][100]['F']

In [7]:
tmlu_dataset["test"][100]

{'id': 'AST_chinese_test-107-3',
 'question': '下列各篇內容與其所屬文體，敘述最適當的是：',
 'A': '〈諫逐客書〉：李斯揣摩秦王心理，陳述逐客之弊，屬奏疏體',
 'B': '〈勸和論〉：鄭用錫為避免械鬥，代官府勸導百姓，屬上對下的詔令體',
 'C': '〈師說〉：韓愈追述儒道先師，屬探究事物本源的論辨體',
 'D': '〈諫太宗十思疏〉：魏徵逐一評述太宗所提的十種治道，屬注疏體',
 'E': None,
 'F': None,
 'answer': 'A',
 'explanation': '',
 'metadata': {'timestamp': '2023-10-09T18:27:20.310857',
  'source': 'AST chinese - 107',
  'explanation_source': ''},
 'human_evaluation': {'quality': '', 'comments': ''},
 'subject': 'AST_chinese'}

In [5]:
from datasets import load_dataset, get_dataset_config_names
dataset_name = "miulab/tmlu"
config="AST_chinese"
cache_dir = 'c:\\Users\\l501l\\Desktop\\TaiwanGPT\\data\\eval'
load_dataset(dataset_name, config, cache_dir=cache_dir)

DatasetDict({
    test: Dataset({
        features: ['id', 'question', 'A', 'B', 'C', 'D', 'E', 'F', 'answer', 'explanation', 'metadata', 'human_evaluation'],
        num_rows: 126
    })
    dev: Dataset({
        features: ['id', 'question', 'A', 'B', 'C', 'D', 'E', 'F', 'answer', 'explanation', 'metadata', 'human_evaluation'],
        num_rows: 5
    })
})

## Token Count

In [1]:
import tiktoken

In [2]:
def num_tokens_from_messages(messages, model="gpt-4o-mini"):
    """Return the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model in {
        "gpt-4o-2024-08-06",
        "gpt-4o-mini",
        "gpt-4o-mini-2024-07-18"
        }:
        tokens_per_message = 3
        tokens_per_name = 1
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}."""
        )
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens

In [ ]:
from openai import OpenAI
import os

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as env var>"))

example_messages = [
    {
        "role": "system",
        "content": "You are a helpful, pattern-following assistant that translates corporate jargon into plain English.",
    },
    {
        "role": "system",
        "name": "example_user",
        "content": "New synergies will help drive top-line growth.",
    },
    {
        "role": "system",
        "name": "example_assistant",
        "content": "Things working well together will increase revenue.",
    },
    {
        "role": "system",
        "name": "example_user",
        "content": "Let's circle back when we have more bandwidth to touch base on opportunities for increased leverage.",
    },
    {
        "role": "system",
        "name": "example_assistant",
        "content": "Let's talk later when we're less busy about how to do better.",
    },
    {
        "role": "user",
        "content": "This late pivot means we don't have time to boil the ocean for the client deliverable.",
    },
]

for model in [
        "gpt-4o-2024-08-06",
        "gpt-4o-mini",
        "gpt-4o-mini-2024-07-18"
    ]:
    print(model)
    # example token count from the function defined above
    print(f"{num_tokens_from_messages(example_messages, model)} prompt tokens counted by num_tokens_from_messages().")
    # example token count from the OpenAI API
    response = client.chat.completions.create(model=model,
    messages=example_messages,
    temperature=0,
    max_tokens=1)
    print(f'{response.usage.prompt_tokens} prompt tokens counted by the OpenAI API.')
    print()